<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/long-epochs/Dacon_satellite_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import random
import sys
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout, MaxPooling2D, BatchNormalization, concatenate, Input
from tensorflow.keras import Model

In [0]:
# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

## 데이터 받아오기

In [0]:
import os.path

if not os.path.isfile('train.npy'):
    !cp '/content/drive/My Drive/2020 Kaggle Study/data/train.npy' train.npy
if not os.path.isfile('test.npy'): 
    !cp '/content/drive/My Drive/2020 Kaggle Study/data/test.npy' test.npy
if not os.path.isfile('gmi_preci.npy'):
    !cp '/content/drive/My Drive/2020 Kaggle Study/data/gmi_preci.npy' gmi_preci.npy

train = np.load('train.npy')  # float32
test = np.load('test.npy')  # float64
gmi_preci = np.load('gmi_preci.npy')  # float32

In [0]:
gmi_preci = np.load('/content/drive/My Drive/2020 Kaggle Study/data/gmi_preci.npy')  # float32

## Data preprocess

In [0]:
# -9999를 포함한 이미지 제거
is_valid = (train[:,:,:,-1].reshape(-1, 1600) < 0).sum(axis=1) == 0
train = train[is_valid]  # (75957, 40, 40, 15)
gmi_preci = gmi_preci[is_valid]  # (75957, 40, 40, 1)

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train[:,:,:,:9].reshape(-1, 9))

def preprocess(data):
    # Normalize sensor data
    data[:,:,:,:9] = scaler.transform(data[:,:,:,:9].reshape(-1, 9)).reshape(-1, 40, 40, 9)

    # Land type
    land_type_data = data[:,:,:,9]
    data[:,:,:,9] = np.where(land_type_data//100 == 2, 0.8,
                              np.where(land_type_data//100 == 3, 0.1,
                                       land_type_data//100))

    return data

In [0]:
x_train = preprocess(train)[:,:,:,:10]
y_train = gmi_preci.reshape(-1, 40, 40)
del train

## Train test split

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.025, random_state=7777)

## 모델만들기

In [0]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [0]:
def build_model(input_layer, start_neurons):
    # 40 x 40 -> 20 x 20
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(input_layer)
    conv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(conv1)
    pool1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((2, 2))(pool1)
    pool1 = Dropout(0.25)(pool1)

    # 20 x 20 -> 10 x 10
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(pool1)
    conv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(conv2)
    pool2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((2, 2))(pool2)
    pool2 = Dropout(0.25)(pool2)

    # 10 x 10 
    convm = Conv2D(start_neurons * 4, (3, 3), activation="relu", padding="same")(pool2)

    # 10 x 10 -> 20 x 20
    deconv2 = Conv2DTranspose(start_neurons * 2, (3, 3), strides=(2, 2), padding="same")(convm)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.25)(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = Conv2D(start_neurons * 2, (3, 3), activation="relu", padding="same")(uconv2)
    uconv2 = BatchNormalization()(uconv2)

    # 20 x 20 -> 40 x 40
    deconv1 = Conv2DTranspose(start_neurons * 1, (3, 3), strides=(2, 2), padding="same")(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.25)(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = Conv2D(start_neurons * 1, (3, 3), activation="relu", padding="same")(uconv1)
    uconv1 = BatchNormalization()(uconv1)
    uconv1 = Dropout(0.25)(uconv1)
    output_layer = Conv2D(1, (1, 1), padding="same", activation='relu')(uconv1)

    return output_layer

input_layer = Input((40, 40, 10))
output_layer = build_model(input_layer, 32)

In [0]:
model = Model(input_layer, output_layer)
model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])

## 모델 학습

In [0]:
model.fit(x_train, y_train, epochs=50, batch_size=256, validation_data=(x_val, y_val))

Epoch 1/50
290/290 [==============================] - 83s 285ms/step - loss: 0.1242 - maeOverFscore_keras: 2.9955 - fscore_keras: 0.5581 - val_loss: 0.1174 - val_maeOverFscore_keras: 2.5568 - val_fscore_keras: 0.5912
Epoch 2/50
290/290 [==============================] - 82s 281ms/step - loss: 0.1016 - maeOverFscore_keras: 1.8959 - fscore_keras: 0.6728 - val_loss: 0.1044 - val_maeOverFscore_keras: 1.9462 - val_fscore_keras: 0.6796
Epoch 3/50
290/290 [==============================] - 83s 285ms/step - loss: 0.0975 - maeOverFscore_keras: 1.7347 - fscore_keras: 0.7017 - val_loss: 0.1000 - val_maeOverFscore_keras: 1.7553 - val_fscore_keras: 0.7163
Epoch 4/50
290/290 [==============================] - 83s 285ms/step - loss: 0.0953 - maeOverFscore_keras: 1.6732 - fscore_keras: 0.7132 - val_loss: 0.0970 - val_maeOverFscore_keras: 1.6135 - val_fscore_keras: 0.7399
Epoch 5/50
290/290 [==============================] - 81s 279ms/step - loss: 0.0936 - maeOverFscore_keras: 1.6130 - fscore_keras: 0.

In [0]:
## 모델 저장하기
model.save('model1901.h5')
!cp model1901.h5 '/content/drive/My Drive/2020 Kaggle Study/model'

## Precipatation GMI -> DPR

In [0]:
dr = [(-1, -1), (-1, 0), (-1, 1),
      (0, -1), (0, 0), (0, 1),
      (1, -1), (1, 0), (1, 1)]

# p1, p2: shape=(-1, 2).
def get_dist(p1, p2):
    x1 = np.deg2rad(p1[:,0])
    y1 = np.deg2rad(p1[:,1])
    x2 = np.deg2rad(p2[:,0])
    y2 = np.deg2rad(p2[:,1])
    dlon = x2 - x1
    dlat = y2 - y1
    a = np.sin(dlat/2)**2 + np.cos(y1) * np.cos(y2) * np.sin(dlon/2)**2 
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))    
    return 6373.0 * c  # km, shape=(-1).

# ori_ll에서의 value 값을 tgt_ll에 대한 값으로 바꾼다.
# value: (40, 40, -1)
# ori_ll, tgt_ll: (40, 40, 2)
# ori_ll: value에 대응되는 원래 latitude, longitude.
# tgt_ll: 변환되는 값에 대응되는 latitude, longitude.
def compen_ll(value, ori_ll, tgt_ll):
    ret = np.empty_like(value)

    n = value.shape[0]
    m = value.shape[1]

    for i in range(n):
        for j in range(m):
            nears = []  # (row, col, value)
            for k in range(9):
                ii = i + dr[k][0]
                jj = j + dr[k][1]
                if ii >= 0 and ii < n and jj >= 0 and jj < m:
                    nears.append((ori_ll[ii, jj][0], ori_ll[ii, jj][1],
                                  tgt_ll[i, j][0], tgt_ll[i, j][1],
                                  value[ii, jj]))

            nears = np.array(nears)  # shape=(-1, 5)
            dists = get_dist(nears[:, 0:2], nears[:, 2:4]).reshape(-1, 1)
            values = nears[:, 4].reshape(-1, 1)
            nears = np.concatenate((dists, values), 1)
            nears = nears[np.argsort(nears[:, 0])]  # sort by dist
            nears = nears[:4, :]  # 가까운 점 4개만 고려

            weights = 1 / (nears[:, 0] ** 2 + sys.float_info.epsilon)
            weighted_sum = (weights * nears[:, 1]).sum()
            ret[i, j] = weighted_sum / weights.sum()
    return ret

In [0]:
x_test = preprocess(test)
pred = model.predict(x_test[:,:,:,:10]).reshape(-1, 40, 40, 1)
test = np.concatenate((x_test, pred), axis=3)

In [0]:
from multiprocessing import Process, Manager

n_procs = 4
procs = []
manager = Manager()
dpr_preci = manager.list([None] * n_procs)

# split test into n_procs arrays
n_imgs = test.shape[0]
splitted = np.split(test, np.arange((n_imgs + n_procs - 1) // n_procs, n_imgs, n_imgs // n_procs))

def proc_func(proc_id):
    part = splitted[proc_id]
    arr = np.empty_like(part[:, :, :, 14])  # shape=(-1, 40, 40)
    for i in range(part.shape[0]):
        arr[i, :, :] = compen_ll(part[i, :, :, 14],
                                 part[i, :, :, 12:14],
                                 part[i, :, :, 10:12])
        if (i + 1) % 1000 == 0:
            print(proc_id, i + 1)
    dpr_preci[proc_id] = arr

for proc_id in range(n_procs):
    proc = Process(target=proc_func, args=(proc_id, ))
    proc.start()
    procs.append(proc)

for proc in procs:
    proc.join()

dpr_preci = np.concatenate(dpr_preci)

## submission 만들기

In [0]:
submission = pd.read_csv('/content/drive/My Drive/2020 Kaggle Study/data/sample_submission.csv')
submission.iloc[:,1:] = dpr_preci.reshape(-1, 1600)

In [0]:
# 제출 파일 저장하기
submission.to_csv('submission1901.csv', index=False)
!cp submission1901.csv '/content/drive/My Drive/2020 Kaggle Study/submission'